# Initialization

In [ ]:
# %load init.ipy
%reload_ext autoreload
%autoreload 2

import os, sys

import numpy as np
import scipy as sp
import scipy.integrate
import matplotlib.pyplot as plt
import matplotlib as mpl

CWD = os.path.abspath(os.path.curdir)
print("CWD: '{}'".format(CWD))

ODIR = os.path.join(CWD, "output", "")
if not os.path.exists(ODIR):
    os.makedirs(ODIR)
    print("Created output directory: '{}'".format(ODIR))

par_dir = os.path.join(CWD, os.path.pardir)
if par_dir not in sys.path:
    sys.path.append(par_dir)
    print("Added parent directory: '{}'".format(par_dir))

import bhem
import bhem.basics
import bhem.utils
import bhem.disks
import bhem.radiation
import bhem.spectra
from bhem.constants import MSOL, H_PLNK, K_BLTZ, SPLC, MPRT, MELC, QELC, BANDS, SIGMA_SB, NWTG

np.seterr(over='ignore');

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times']})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

FS_TITLE = 20
FS_LABEL = 16

plt.rcParams.update({'axes.titlesize': FS_TITLE})
plt.rcParams.update({'axes.labelsize': FS_LABEL})
plt.rcParams.update({'xtick.labelsize': FS_LABEL})
plt.rcParams.update({'ytick.labelsize': FS_LABEL})


## Parameters

In [ ]:
MASS = 1e7 * MSOL
FEDD = 0.1

PATH_OUTPUT = os.path.join(ODIR, 'shakura-sunyaev', '')

if not os.path.exists(PATH_OUTPUT):
    os.makedirs(PATH_OUTPUT)

In [ ]:
thin = bhem.disks.Thin(MASS, fedd=FEDD)

### Derived

In [ ]:
mdot = bhem.basics.eddington_accretion(MASS)
rsch = bhem.basics.radius_schwarzschild(MASS)
# rads = np.logspace(np.log10(6), 4, 200) * rsch
rads = thin.rads
freqs = np.logspace(10, 18, 120)

# Disk Primitives Profiles

In [ ]:
# temp = bhem.basics.temperature_profile(MASS, mdot, rads)

In [ ]:
mu = 1.2
pres_over_dens = (K_BLTZ * thin.temp / (mu * MPRT)) + (4*SIGMA_SB*thin.temp**4 / (3*SPLC) )
hh = np.sqrt(pres_over_dens * 2 * (thin.rads**3) / (NWTG * thin.mass))

In [ ]:
fig, ax = plt.subplots(figsize=[6, 4])
ax.set(xscale='log', yscale='log')

ax.plot(thin.rads, hh/thin.rads)
IND = 1/8
norm = hh[0]/thin.rads[0]
ax.plot(thin.rads, np.power(thin.rads/thin.rads[0], IND) * norm, 'k--')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=[10, 5])
ax.set(xscale='log', xlabel='Radius [$R_s$]', yscale='log', ylabel='Temperature [K]')

ax.plot(rads/rsch, thin.temp, 'r-', lw=2.0, alpha=0.8)
plt.show()

# Blackbody Spectrum

In [ ]:
# erg/s/Hz/cm^2/steradian
# bb_spec_rad = bhem.basics.blackbody_spectral_radiance(MASS, mdot, rads[:, np.newaxis], freqs[np.newaxis, :])
rr = rads[np.newaxis, :]
ff = freqs[:, np.newaxis]
bb_spec_rad = thin._blackbody_spectral_radiance(rr, ff)

In [ ]:
xx, yy = np.meshgrid(rr, ff)

norm = mpl.colors.LogNorm(vmin=1e-10, vmax=np.max(bb_spec_rad))
smap = mpl.cm.ScalarMappable(norm=norm, cmap='hot')
smap.cmap.set_under('0.5')

fig, axes = plt.subplots(figsize=[14, 6], ncols=2)
for ax in axes:
    ax.set(xscale='log', xlabel='Radius [$R_s$]', yscale='log', ylabel='Freq [Hz]')
    for nn, band in bhem.constants.BANDS.items():
        ax.axhline(band.freq, color=band.color, lw=2.0, alpha=0.5)

pcm = axes[0].pcolormesh(xx/rsch, yy, bb_spec_rad, norm=norm, cmap=smap.cmap)
plt.colorbar(pcm, ax=axes[0], orientation='horizontal')

finds = (1e14 < freqs) & (freqs < 1e16)

norm = mpl.colors.Normalize(0.0, np.max(bb_spec_rad[finds, :]))
smap = mpl.cm.ScalarMappable(norm=norm, cmap='hot')
pcm = axes[1].pcolormesh(xx[finds, :]/rsch, yy[finds, :], bb_spec_rad[finds, :], norm=norm, cmap=smap.cmap)
plt.colorbar(pcm, ax=axes[1], orientation='horizontal')

plt.show()

In [ ]:
# bb_lum = bhem.basics.blackbody_spectral_luminosity(MASS, mdot, freqs)
bb_lum = thin.blackbody_spectral_luminosity(freqs)

In [ ]:
fig, ax = plt.subplots(figsize=[10, 5])
ax.set(xscale='log', xlabel='Frequency [Hz]',
       yscale='log', ylabel='Spectral Luminosity [erg/s/Hz]', ylim=[1e20, 1e30])

ax.plot(freqs, bb_lum, 'r-', lw=2.0, alpha=0.6)

for nn, band in bhem.constants.BANDS.items():
    ax.axvline(band.freq, color=band.color, lw=1.0, alpha=0.5)

plt.show()

# Varying Eddington Ratios : Spectra and Efficiencies

In [ ]:
_MASS = 1e9 * MSOL

fig, axes = plt.subplots(figsize=[12, 5], ncols=2)
plt.subplots_adjust(wspace=0.55, left=0.08, right=0.92, top=0.96)

for ax in axes:
    ax.set(xscale='log', yscale='log')
    ax.grid(True, which='major', axis='both', c='0.5', alpha=0.5)


ax = axes[0]
ax.set(xlabel='Frequency [Hz]', # xlim=[1e5, 1e22], 
       ylabel='$\\nu \, F_\\nu [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Cumulative Luminosity $[\mathrm{erg \,\, s}^{-1}]$')

fedds = np.logspace(-6, 0, 7)[::-1]
lums = np.zeros_like(fedds)

cmap = mpl.cm.get_cmap('gist_heat_r')
colors = [cmap(xx) for xx in np.linspace(0.1, 0.9, fedds.size)]
ymax = 0.0

for ii, fe in enumerate(fedds):
    label = '${:+.1f}$'.format(np.log10(fe))
    cc = colors[ii]
    kw = dict(color=cc, lw=2.0, label=label)

    _thin = bhem.disks.Thin(_MASS, 100, fedd=fe)
    bb_lum = _thin.blackbody_spectral_luminosity(freqs)
    lum = bb_lum
    
    ax.plot(freqs, freqs*lum, ls='--', alpha=0.5, **kw)
    ymax = np.maximum(np.max(freqs*lum), ymax)
    
    lum_mid = bhem.utils.log_midpoints(lum)
    freqs_mid = bhem.utils.log_midpoints(freqs)
    df = np.diff(freqs)
    cumlum = np.cumsum(df * lum_mid)
    lums[ii] = cumlum[-1]
    tw.plot(freqs_mid, cumlum, alpha=0.8, **kw)    
    
tw.set_ylim([1e32, 1e50])
ax.set_ylim([1e30, 3*ymax])
ax.text(0.02, 0.98, "$M = {:.1e} \,\, M_\odot$".format(_MASS/MSOL), transform=ax.transAxes,
         ha='left', va='top')
    
for nn, band in bhem.constants.BANDS.items():
    ax.axvline(band.freq, color=band.color, lw=1.0, alpha=0.5)

ax.legend(title="$\log(\dot{M}/\dot{M}_\mathrm{edd})$", fontsize=12, loc='center left')


ax = axes[1]
ax.set(xlabel='Eddington Fraction', 
       ylabel='$L_\mathrm{bol} [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Efficiency')

mdot_edd = bhem.basics.eddington_accretion(_MASS)
effs = lums/(mdot_edd * fedds * SPLC**2)

ax.plot(fedds, lums, 'r-', alpha=0.8)
tw.plot(fedds, effs, 'r--', alpha=0.8)
tw.plot(fedds, np.minimum(10*fedds, 0.1), color='0.5', ls='--', alpha=0.5)

plt.show()

fname = 'lum-eff_thin_mdot'
fname = os.path.join(PATH_OUTPUT, fname)
fig.savefig(fname + '.pdf')
fig.savefig(fname + '.png')
print("Saved to '{}'".format(fname))

# Disk Truncation

In [ ]:
_MASS = 1e6 * MSOL
_FEDD = 1e-1
VAR_LABEL = "$\log(R_\mathrm{max}/R_s)$"
BAND = "v"
NRAD = 100

fig, axes = plt.subplots(figsize=[12, 5], ncols=2)
plt.subplots_adjust(wspace=0.55, left=0.08, right=0.92, top=0.96)

for ax in axes:
    ax.set(xscale='log', yscale='log')
    ax.grid(True, which='major', axis='both', c='0.5', alpha=0.5)


ax = axes[0]
ax.set(xlabel='Frequency [Hz]', # xlim=[1e5, 1e22], 
       ylabel='$\\nu \, F_\\nu [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Cumulative Luminosity $[\mathrm{erg \,\, s}^{-1}]$')

# fedds = np.logspace(-6, 0, 7)[::-1]
rad_max = np.logspace(1, 5, 9)
lums = np.zeros_like(rad_max)
lums_spec = np.zeros_like(rad_max)

cmap = mpl.cm.get_cmap('gist_heat_r')
colors = [cmap(xx) for xx in np.linspace(0.1, 0.9, rad_max.size)]
ymax = 0.0

for ii, rm in enumerate(rad_max):
    label = '${:.1f}$'.format(np.log10(rm))
    cc = colors[ii]
    kw = dict(color=cc, lw=2.0, label=label)

    _thin = bhem.disks.Thin(_MASS, fedd=_FEDD, rmax=rm, nrad=NRAD)
    bb_lum = _thin.blackbody_spectral_luminosity(freqs)
    lum = bb_lum
    
    ax.plot(freqs, freqs*lum, ls='--', alpha=0.5, **kw)
    ymax = np.maximum(np.max(freqs*lum), ymax)
    
    _slum = bhem.utils.log_interp1d(freqs, lum*freqs)(BANDS[BAND].freq)
    lums_spec[ii] = _slum
    
    lum_mid = bhem.utils.log_midpoints(lum)
    freqs_mid = bhem.utils.log_midpoints(freqs)
    df = np.diff(freqs)
    cumlum = np.cumsum(df * lum_mid)
    lums[ii] = cumlum[-1]
    tw.plot(freqs_mid, cumlum, alpha=0.8, **kw)    
    
tw.set_ylim([1e32, 1e50])
ax.set_ylim([1e30, 3*ymax])
ax.text(0.02, 0.98, "$M = {:.1e} \,\, M_\odot$".format(_MASS/MSOL), transform=ax.transAxes,
         ha='left', va='top')
    
for nn, band in bhem.constants.BANDS.items():
    ax.axvline(band.freq, color=band.color, lw=1.0, alpha=0.5)

ax.legend(title=VAR_LABEL, fontsize=12, loc='center left')


ax = axes[1]
ax.set(xlabel=VAR_LABEL, 
       ylabel='$L_\mathrm{bol} [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Efficiency')

mdot_edd = bhem.basics.eddington_accretion(_MASS)
effs = lums/(mdot_edd * _FEDD * SPLC**2)

ax.plot(rad_max, lums, 'r-', alpha=0.8, lw=2.0)
ax.plot(rad_max, lums_spec, 'b-', alpha=0.8)
tw.plot(rad_max, effs, 'r--', alpha=0.8)
# tw.plot(rad_max, np.minimum(10*fedds, 0.1), color='0.5', ls='--', alpha=0.5)

plt.show()

fname = 'spec-eff_thin_rmax'
fname = os.path.join(PATH_OUTPUT, fname)
fig.savefig(fname + '.pdf')
print("Saved to '{}'".format(fname))

In [ ]:
_MASS = 1e7 * MSOL
_FEDD = 1e-1
VAR_LABEL = "$\log(R_\mathrm{max}/R_s)$"
BAND = "v"
RAD_MAX = 1e3

fig, axes = plt.subplots(figsize=[12, 5], ncols=2)
plt.subplots_adjust(wspace=0.55, left=0.08, right=0.92, top=0.96)

for ax in axes:
    ax.set(xscale='log', yscale='log')
    ax.grid(True, which='major', axis='both', c='0.5', alpha=0.5)


ax = axes[0]
ax.set(xlabel='Frequency [Hz]', # xlim=[1e5, 1e22], 
       ylabel='$\\nu \, F_\\nu [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Cumulative Luminosity $[\mathrm{erg \,\, s}^{-1}]$')

# fedds = np.logspace(-6, 0, 7)[::-1]
rad_max = np.logspace(1, 5, 8)
lums = np.zeros_like(rad_max)
lums_spec = np.zeros_like(rad_max)

cmap = mpl.cm.get_cmap('gist_heat_r')
colors = [cmap(xx) for xx in np.linspace(0.1, 0.9, rad_max.size)]
ymax = 0.0

for ii, rm in enumerate(rad_max):
    label = '${:.1f}$'.format(np.log10(rm))
    cc = colors[ii]
    kw = dict(color=cc, lw=2.0, label=label)

    _thin = bhem.disks.Thin(_MASS, fedd=_FEDD, rmax=rm, nrad=NRAD)
    bb_lum = _thin.blackbody_spectral_luminosity(freqs)
    lum = bb_lum
    
    ax.plot(freqs, freqs*lum, ls='--', alpha=0.5, **kw)
    ymax = np.maximum(np.max(freqs*lum), ymax)
    
    _slum = bhem.utils.log_interp1d(freqs, lum*freqs)(BANDS[BAND].freq)
    lums_spec[ii] = _slum
    
    lum_mid = bhem.utils.log_midpoints(lum)
    freqs_mid = bhem.utils.log_midpoints(freqs)
    df = np.diff(freqs)
    cumlum = np.cumsum(df * lum_mid)
    lums[ii] = cumlum[-1]
    tw.plot(freqs_mid, cumlum, alpha=0.8, **kw)    
    
tw.set_ylim([1e32, 1e50])
ax.set_ylim([1e30, 3*ymax])
ax.text(0.02, 0.98, "$M = {:.1e} \,\, M_\odot$".format(_MASS/MSOL), transform=ax.transAxes,
         ha='left', va='top')
    
for nn, band in bhem.constants.BANDS.items():
    ax.axvline(band.freq, color=band.color, lw=1.0, alpha=0.5)

ax.legend(title=VAR_LABEL, fontsize=12, loc='center left')


ax = axes[1]
ax.set(xlabel=VAR_LABEL, 
       ylabel='$L_\mathrm{bol} [\mathrm{erg \,\, s}^{-1}]$')
tw = ax.twinx(); tw.set(yscale='log', ylabel='Efficiency')

mdot_edd = bhem.basics.eddington_accretion(_MASS)
effs = lums/(mdot_edd * _FEDD * SPLC**2)

ax.plot(rad_max, lums, 'r-', alpha=0.8, lw=2.0)
ax.plot(rad_max, lums_spec, 'b-', alpha=0.8)
tw.plot(rad_max, effs, 'r--', alpha=0.8)
# tw.plot(rad_max, np.minimum(10*fedds, 0.1), color='0.5', ls='--', alpha=0.5)

plt.show()

fname = 'spec-eff_thin_rmax'
fname = os.path.join(PATH_OUTPUT, fname)
fig.savefig(fname + '.pdf')
print("Saved to '{}'".format(fname))